# Análisis de datos NGS

### ¿Que es NGS?

Next Generation sequencing o NGS es un conjunto de técnicas de ultra secuenciación genómica paralela, las cuales incluyen WGS (secuenciación del genoma completo), WES (secuenciacion del exoma completo) y RNAseq (secuenciacion del RNA), Dependiendo del propósito de nuestro estudio podemos usar un tipo de técnicas o varias. En este minitutorial nos centraremos en analizar datos de exoma, WES.


<table class="image">
<tr><td><img src= "imagenes/descarga.jpg" width="1000" height="1000"></td></tr>
</table>

### ¿Qué tipo de análisis se llevará a cabo?

En este minitutorial llevaremos a cabo el análisis de muestras de exoma para la búsqueda de mutaciones SNV (Single Nucleotide Variant) e Indels(Inserciones o deleciones).

Un ejemplo de porque es tan útil este tipo de análisis es por ejemplo la búsqueda de marcadores para terapias contra el cáncer.


<table class="image">
<tr><td><img src= "imagenes/tipos.png" width="600" height="600"></td></tr>

### ¿Cuáles son los pasos a seguir para el análisis en este minitutorial?

1. Instalar software básico necesario para el análisis
    - Instalar Ubuntu, Windows Subsystem for Linux 2 (WSL2) y conda
    - Clonar GitHub repo
    - Crear environment en conda con las herramientas necesarias
2. Descarga de las bases de datos y exoma para el análisis
    - Obtener las referencias
    - Obtener datos genómicos
3. Análisis de exoma:
    - Datos de input
    - Control de calidad: Estudiamos la calidad de los datos que han sido secuenciados
    - Trimming: Solucionamos ciertos problemas de calidad de los datos
    - Alineamiento: Alineamos las muestras con el genoma de referencia
    - Preprocesado: Procesamos los datos para prepararlos para la búsqueda de mutaciones
    - Variant Calling: Buscamos las mutaciones

### 1. Instalar software básico necesario para el análisis

Este análisis se llevará a cabo usando el comando de línea de Linux o Mac y en el caso de Windows en una máquina virtual de Ubuntu para Linux.

#### Instalar Ubuntu, Windows Subsystem for Linux 2 (WSL2) y conda

- Descargad Ubuntu 20.04 for Windows

- Busca Ubuntu en el menú de búsqueda de Windows y abra la terminal de Linux.

- Crea tu usuario y contraseña.

- Opcional: Para instalar WSL, siga las instrucciones en [https://docs.microsoft.com/en-us/windows/wsl/install](https://docs.microsoft.com/en-us/windows/wsl/install) .

- Opcional: Copia y pega \\\wsl$ en el exporador de archivos
    
- Descargar miniconda con:

    wget [https://repo.anaconda.com/miniconda/Miniconda3-py38_4.11.0-Linux-x86_64.sh](https://repo.anaconda.com/miniconda/Miniconda3-py38_4.11.0-Linux-x86_64.sh) 
        
    (Comprueba aquí para otros instaladores [ttps://docs.conda.io/en/latest/miniconda.html#linux-installers](https://docs.conda.io/en/latest/miniconda.html#linux-installers))
        
- Instalar miniconda con: bash Miniconda3-latest-Linux-x86_64.sh

- Opcional: Instalar anaconda navigator en este enlace: https://www.anaconda.com/products/distribution
    

#### Clonar Github repo

El siguiente paso es clonar el repositorio que contiene este jupyter notebook y el archivo yml para crear el environment:


In [ ]:
%%bash

# Copiando github repo
git clone https://github.com/robertofg96/sequencingAnalisys.git

#### Crear environment en conda con las herramientas necesarias

Para el análisis del exoma debemos descargar varios paquetes de software, los cuales son;

- ***Fastqc:*** Herramienta utilizada para proporcionar una descripción general de las métricas básicas de control de calidad para datos de secuenciación sin procesar de próxima generación.

- ***Multiqc:*** MultiQC busca registros de análisis en un directorio determinado y compila un informe HTML. Es una herramienta de uso general, perfecta para resumir el resultado de numerosas herramientas bioinformáticas.

- ***BBmap/BBtools:*** Incluye herramientas como bbduk para recortar reads de datos secuenciados de baja calidad, con adaptadores o para solucionar otros problemas de los datos brutos.

- ***Samtools:*** Samtools es un conjunto de programas para interactuar con datos de secuenciación de alto rendimiento.

- ***GATK:*** El GATK es un paquete de software para identificar SNV e indel en datos de ADN y RNAseq de línea germinal.

- ***BWA:*** BWA es un paquete de software para mapear secuencias contra un gran genoma de referencia.

Una forma fácil de instalarlos es crear environment en conda con todas las dependencias usando el archivo yml que tenéis en el repositorio.

In [ ]:
%%bash

# Con esta linea de codigo podeis crear el environment contodas las dependencias para el analisis
conda env create -n ${nombreEnv} -f sequencingEnvironment.yml

In [ ]:
%%bash

# Activar el environment
conda activate ${nombreEnv}

### 2. Descarga de las bases de datos y exoma para el analisis

#### ***Obtener las referencias***

Para llevar a cabo el analisis necesitamos tambien los datos de exoma y bases de datos de referencia del genoma humano mutaciones somaticas. Estas referencias se componene de:

- ***Homo_sapiens_assembly38.fasta:*** Genoma de referencia.

- ***Homo_sapiens_assembly38.known_indels.vcf.gz:*** Base de datos de indels conocidas.

- ***Homo_sapiens_assembly38.dbsnp138.vcf:*** Base de datos de snps.

- ***af-only-gnomad.raw.sites.vcf:*** Base de datos de variantes.

In [ ]:
%%bash

# Crea las carpetas donde guardaremos las referencias
mkdir references references/genome references/indels references/snps references/gnomad

# Descargar referencias

# Descargar genoma
gsutil cp gs://genomics-public-data/references/hg38/v0/Homo_sapiens_assembly38.fasta references/genome/
gsutil cp gs://genomics-public-data/references/hg38/v0/Homo_sapiens_assembly38.dict references/genome/


# Descaragar base de datos de indels
gsutil cp gs://genomics-public-data/references/hg38/v0/Homo_sapiens_assembly38.known_indels.vcf.gz references/indels
gsutil cp gs://genomics-public-data/references/hg38/v0/Homo_sapiens_assembly38.known_indels.vcf.gz.tbi references/indels
unzip references/indels Homo_sapiens_assembly38.known_indels.vcf.gz
unzip references/indels Homo_sapiens_assembly38.known_indels.vcf.gz.tbi


# Descargar base de datos de snps
gsutil cp gs://genomics-public-data/references/hg38/v0/Homo_sapiens_assembly38.dbsnp138.vcf references/snps
gsutil cp gs://genomics-public-data/references/hg38/v0/Homo_sapiens_assembly38.dbsnp138.vcf.idx references/snps


# Descargar base de datos de gnomad
gsutil cp gs://storage.cloud.google.com/gatk-best-practices/somatic-b37/af-only-gnomad.raw.sites.vcf references/gnomad

In [ ]:
%%bash

# Crear un index del genoma
samtools faidx references/genome/Homo_sapiens_assembly38.fasta

#### ***Obtener los datos genomicos***

In [ ]:
%%bash

# Crea la carpete para los samples del analisis
mkdir sample1_file sample1_file/sample1_fastq sample1_file/sample1_fastqc sample1_file/sample1_trimming sample1_file/sample1_alineado sample1_file/sample1_preprocesado sample1_file/sample1_mutect2
cd sample1_file

# Descarga los samples
wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR099/SRR099958/SRR099958_1.fastq.gz
wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR099/SRR099958/SRR099958_2.fastq.gz

# unzip los datos
unzip SRR099958_1.fastq.gz
unzip SRR099958_2.fastq.gz

# cambiar nombres
mv SRR099958_1.fastq sample1_1.fastq
mv SRR099958_2.fastq sample1_2.fastq

# mover los datos a la carpet fastq
mv sample1_*.fastq sample1_fastq

### Analisis de exoma:

#### Datos de input

Los datos genomicos secuenciados vienen en formato fastq.

***Fastq:***

El formato FASTQ es un formato basado en texto para almacenar tanto una secuencia biológica (normalmente una secuencia de nucleótidos) como sus puntuaciones de calidad correspondientes.
<table class="image">
<tr><td><img src= "imagenes/A-sample-of-the-FASTQ-file.png" width="1000" height="1000"></td></tr>
</table>

***Datos de secuanciancion pair y single end***

Cuando secuenciamos podemos hacerlo de dos maneras:
    
***- Single end:*** Implica secuenciar el ADN de un solo extremo. Esta solución ofrece grandes volúmenes de datos de alta calidad, de forma rápida y económica. La secuenciación single end puede ser una buena opción para determinados métodos, como secuenciación de inmunoprecipitación de cromatina (ChIP-Seq) o ARN-Seq pequeño.

***- Pair end:*** Permite secuenciar ambos extremos de un fragmento y generar datos de secuencias de alta calidad. La secuenciación pair end facilita la detección de reordenamientos genómicos y elementos de secuencias repetitivas, así como fusiones de genes.

Además de producir el doble de lecturas por el mismo tiempo y esfuerzo en la preparación de la biblioteca, las secuencias alineadas como pair end permiten una alineación de lectura más precisa y la capacidad de detectar variantes de inserción-eliminación (indel), lo que no es posible con single end.

Para mas informacion: https://www.youtube.com/watch?v=hRKes6_dv2k


<font color='red'>***En este analisis usaremos pair end data***</font>

#### <font color='blue'>***Primer Paso: Control de calidad***</font>

Los secuenciadores modernos de alto rendimiento pueden generar decenas de millones de secuencias de una vez. Antes de analizar esta secuencia para sacar conclusiones biológicas, debemos llevar a cabo controles de calidad simples para asegurarse de que los datos sin procesar se vean bien
y no hay problemas o sesgos en sus datos que puedan afectar al analisis.

FastQC tiene como objetivo proporcionar una forma sencilla de realizar un control de calidad en los datos de secuenciación.

https://rtsf.natsci.msu.edu/genomics/tech-notes/fastqc-tutorial-and-faq/#:~:text=The%20output%20from%20FastQC%2C%20after,or%20%E2%80%9CFail%E2%80%9D%20is%20assigned.


<table class="image">
    
<tr><td><img src= "imagenes/fastqc.png" width="600" height="600"></td></tr>

In [ ]:
# Descarga los samples
fastqc sample_file/sample1_1.fastq -o sample_file/sample1_fastqc/
fastqc sample_file/sample1_2.fastq -o sample_file/sample1_fastqc/

#### <font color='blue'>***Segundo Paso: Trimming***</font>

El trimado es un proceso que no siempre hay que llevar a cabo y depende de los resultados del control de calidad. Los softwares de trimado suelen usarse para mejorar la calidade de los reads, quitar adaptadores, resolver problemas de contaminacion entre otros.

En nuestro caso usaremos bbduk para llevar a cabo el trimado. En estos enlaces teneis mas informacion sobre todo lo que podemos hacer con esta herramienta.

https://jgi.doe.gov/data-and-tools/software-tools/bbtools/bb-tools-user-guide/bbduk-guide/

http://seqanswers.com/forums/showthread.php?t=42776

In [ ]:
%%bash

# Haciendo trimmado
bbduk.sh in=sample1_fastq/sample1_1.fastq in2=sample1_fastq/sample1_2.fastq out=sample1_trimmed_1.fastq.gz out2=sample1_trimmed_2.fastq.gz qtrim=rl trimq=30

#### <font color='blue'>***Tercer Paso: Alineamineto***</font>

A continuacion necesitamos llevar a cabo el alineamiento/mapeado con el genoma de referencia.

La secuenciación produce una colección de secuencias sin contexto genómico. No sabemos a qué parte del genoma corresponden las secuencias. Mapear las lecturas de un experimento a un genoma de referencia es un paso clave en el análisis de datos genómicos modernos. Con el mapeo, las lecturas se asignan a una ubicación específica en el genoma.

Cuando llevemos a cabo el mapeado nuestro output sera un archivo BAM:

***Bam files:***

Un archivo BAM (*.bam) es la versión binaria comprimida de un archivo SAM que se utiliza para representar secuencias alineadas. Los formatos SAM y BAM se describen en detalle en https://samtools.github.io/hts-specs/SAMv1.pdf.

Los archivos BAM contienen una sección de encabezado y una sección de alineación:

▶ Encabezado: contiene información sobre todo el archivo, como el nombre de la muestra, la longitud de la muestra y el método de alineación. Las alineaciones en la sección de alineaciones están asociadas con información específica en la sección de encabezado.

▶ Alineaciones: contiene el nombre, la secuencia y la calidad de los reads, ademas tambien contine la información de alineación y algunas etiquetas. El nombre de reads incluye el cromosoma, la coordenada de inicio y la calidad de la alineación.

Para el alineamiento usaremos BWA. Podeis encontrar mas infomracion en este link:

http://bio-bwa.sourceforge.net/

<table class="image">
<tr><td><img src= "imagenes/650px-BamRecNew.png" width="1000" height="1000"></td></tr>
</table>

En primer lugar debemos crear un index usando como referencia el genoma con el que vamos a alinear:

In [ ]:
%%bash

# Crear el index de BWA
bwa index references/genome/Homo_sapiens_assembly38.fasta

Una vez tenemos el index podemos alinear nuestras muestras

In [ ]:
%%bash

# Alinear las muestras con el genoma de referencia
bwa mem -M -R "@RG\tID:sample1\tSM:Normal\tLB:exome\tPL:ILLUMINA" references/BWAIndex \
sample1_file/sample1_trimmed/sample1_trimmed_1.fastq.gz sample1_file/sample1_trimmed/sample1_trimmed_2.fastq.gz > sample1_file/sample1_aligned/sample1.sam

#### <font color='blue'>***Cuarto paso: Preprocesado***</font>

EL preprocesado tiene diferentes pasos:

- Convertir la muestra alineada de formato SAM a BAM

- Ordenar por coordenadas la muestra alineada

- Marcado de duplicados

- Crear el index de la muestra alineada

- BaseRecalibrator se usa para crear una tabla basada en covariables específicas. Realiza un recorrido por locus operando solo en sitios que están en los sitios conocidos VCF. Los recursos ExAc, gnomAD o dbSNP se pueden utilizar como sitios de variación conocidos. 

El objetivo de este procedimiento es corregir el sesgo sistemático que afecta la asignación de puntuaciones de calidad base por parte del secuenciador. El primer paso consiste en calcular el error empíricamente y encontrar patrones en la forma en que el error varía con las funciones de basecall en todas las bases. Las observaciones relevantes se escriben en una tabla de recalibración. El segundo paso consiste en aplicar correcciones numéricas a cada basecall individual según los patrones identificados en el primer paso (registrados en la tabla de recalibración) y escribir los datos recalibrados en un nuevo archivo BAM o CRAM.

In [ ]:
%%bash

# Sam a BAM
picard SamFormatConverter INPUT=sample1_file/sample1_aligned/sample1.sam OUTPUT=sample1_file/sample1_preprocesado/sample1.bam VALIDATION_STRINGENCY=LENIENT

# Ordenar por coordenadas genticas
picard SortSam SORT_ORDER=coordinate INPUT=sample1_file/sample1_preprocesado/sample1.bam OUTPUT=sample1_file/sample1_preprocesado/sample1_sorted.bam VALIDATION_STRINGENCY=LENIENT

# Marcar duplicados
picard MarkDuplicates METRICS_FILE=sample1_file/sample1_preprocesado/metrics_sample1.txt INPUT=sample1_file/sample1_preprocesado/sample1_sorted.bam OUTPUT=sample1_file/sample1_preprocesado/sample1_dedup.bam VALIDATION_STRINGENCY=LENIENT

# Crear index
picard BuildBamIndex INPUT=sample1_file/sample1_preprocesado/sample1_dedup.bam OUTPUT=sample1_file/sample1_preprocesado/sample1_dedup.bai

# Recalibrado
gatk BaseRecalibrator -I sample1/sample1_preprocesado/sample1_dedup.bam -R references/genome/Homo_sapiens_assembly38.fasta \
--known-sites references/indels/Homo_sapiens_assembly38.known_indels.vcf.gz --known-sites references/snps/Homo_sapiens_assembly38.dbsnp138.vcf -O sample1_file/sample1_preprocesado/sample1_dedup_recal_data.table

# Aplicar recalibrado
gatk ApplyBQSR -I sample1_file/sample1_preprocesado/sample1_dedup.bam -R  references/genome/Homo_sapiens_assembly38.fasta \
--bqsr-recal-file sample1_file/sample1_preprocesado/sample1_dedup_recal_data.table -O sample1_file/sample1_preprocesado/sample1_recal.bam

#### <font color='blue'>***Quinto paso: Variant Calling***</font>


Variant calling es un tipo de software que nos permite escanear muestras genomicas en busca de mutaciones. El software que usamos nos permite buscar varios tipos de mutaciones y ademas podemos filtrar las mutaciones germinales de por ejemplo muestras tumorales si introducimos muestras de tejido normal.

Este tipo de herramientas propocionan como output archivos vcf. El VCF especifica el formato de un archivo de texto utilizado en bioinformática para almacenar variaciones de secuencias de genes.

<tr><td><img src= "imagenes/VCF.png" width="1000" height="1000"></td></tr>

In [ ]:
%%bash

# Somaticas (no he usado las mismas muestras)
gatk Mutect2 -R references/genome/Homo_sapiens_assembly38.fasta -I sample1_file/sample1_preprocesado/sample2_recal_tumor.bam -I sample1_file/sample2_preprocesado/sample1_recal_normal.bam \
--germline-resource references/gnomad/af-only-gnomad.raw.sites.vcf -O sample1_som_vars_mutect2_apareado.vcf.gz 

***Opcional: Multiqc***

Multiqc es una herramienta que permite juntar de controles de calidad, estadisticas del alineamiento y busqueda de mutaciones en un report congraficos interactivos.

In [ ]:
%%bash

multiqc SRR099958_fastqc/*_fastqc.zip